## Poisson based greedy algorithm to identify active radionuclides

In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from script_unmix.unmix_algo import chambolle_pock
from script_unmix.select_rn import ompPoisson
from utils.operators import prox_Nonnegative,Lpoisson,dual_prox,grad_LS

## Dictionary of spectral signatures 
- the dictionary contains 10 radionuclides that may composed a measured spectrum
- each spectrum is composed of 16173 channels

In [2]:
num = 16173
phi_dic = np.zeros((num,10))
phi_dic[:,0] = sio.loadmat('synthetic_data/Be7.mat')['counts']
phi_dic[:,1] = sio.loadmat('synthetic_data/Na22.mat')['counts']
phi_dic[:,2] = sio.loadmat('synthetic_data/K40.mat')['counts']
phi_dic[:,3] = sio.loadmat('synthetic_data/Cs137.mat')['counts']
phi_dic[:,4] = sio.loadmat('synthetic_data/Pb210.mat')['counts']
phi_dic[:,5] = sio.loadmat('synthetic_data/Tl208.mat')['counts']
phi_dic[:,6] = sio.loadmat('synthetic_data/Bi212.mat')['counts']
phi_dic[:,7] = sio.loadmat('synthetic_data/Pb212.mat')['counts']
phi_dic[:,8] = sio.loadmat('synthetic_data/Bi214.mat')['counts']
phi_dic[:,9] = sio.loadmat('synthetic_data/Pb214.mat')['counts']

## Generate simulated gamma spectrum:
- 5 radionuclides + a background spectrum
- the measured spectrum is composed of 16173 channels

In [3]:
num = 16173
phi_sig = np.zeros((num,5))
phi_sig[:,0] = sio.loadmat('synthetic_data/Be7.mat')['counts']
phi_sig[:,1] = sio.loadmat('synthetic_data/Na22.mat')['counts']
phi_sig[:,2] = sio.loadmat('synthetic_data/K40.mat')['counts']
phi_sig[:,3] = sio.loadmat('synthetic_data/Cs137.mat')['counts']
phi_sig[:,4] = sio.loadmat('synthetic_data/Pb210.mat')['counts']
b = np.load('synthetic_data/b.npy')
a0 = np.load('synthetic_data/a0.npy')
x = np.random.poisson(phi_sig@a0+b)

## Indentification and quantification of the 5 active radionuclides with Poisson-based OMP algorithm
- the multiplicative update algorithm allows faster convergence


In [4]:
def algo_cp(phi_in,n_item=0,tol=1e-10,verb=True):
    phi = np.copy(phi_in)
    N = np.linalg.norm(np.dot(phi.T,phi))
    phi = phi/N
    L = np.linalg.norm(np.dot(phi.T,phi),ord=2)
    sigma = 1e-4
    tau = (.9/(sigma*L))

    nP = np.shape(phi)
    a_initial = 1./nP[1]*np.ones((nP[1]))
    K = lambda y: np.dot(phi,y)
    KT = lambda y: np.dot(phi.T,y)
    prox_g = prox_Nonnegative
    prox_f = lambda a,step:Lpoisson(a+b,step)(x)-b
    prox_fC = lambda u,sigma: dual_prox(prox_f)(u,sigma)
    a_estim,a_list = chambolle_pock(prox_fC, prox_g, a_initial, K, KT, sigma,tau,n_item=n_item,tol=tol,verb=verb,error_list=False)
    a_estim = a_estim/N
    return a_estim
def algoMulti(Phi,n_item=100,verb=True): 
    nP = np.shape(Phi)
    a = 1./nP[1]*np.ones((nP[1],1))
    diff = np.linalg.norm(a)
    dPhi = np.sum(Phi,axis=0)
    
    for r in range(n_item):
        a_new =a*(np.diag(1./dPhi)@(Phi.T@(x.reshape(-1,1)/(Phi@a + b.reshape(-1,1)))))
        diff = np.linalg.norm(a-a_new)/np.linalg.norm(a)
        if verb:
            print("It. #",r,' - ',diff)
        a = np.copy(a_new)
    return a[:,0]

In [5]:
algo_unmix = lambda z: algoMulti(Phi=z,n_item=1000,verb=False)
RN = ompPoisson(x=x,Phi=phi_dic,bdf=b,alpha=0.1,algo_unmix=algo_unmix).algo_selection()

Deviance :  -4470722.116245281 / -20440068.226957303 - selected this turn :  0
Deviance :  -20440068.226957303 / -20654739.052421935 - selected this turn :  4
Deviance :  -20654739.052421935 / -20662637.993787676 - selected this turn :  1
Deviance :  -20662637.993787676 / -20666905.297516864 - selected this turn :  2
Deviance :  -20666905.297516864 / -20667219.320302106 - selected this turn :  3


In [ ]:
algo_unmix = lambda z: algo_cp(phi_in=z,n_item=20000,tol=1e-10,verb=False)
RN = ompPoisson(x=x,Phi=phi_dic,bdf=b,alpha=0.1,algo_unmix=algo_unmix).algo_selection()